In [ ]:
import os
import sys

import numpy as np
from scipy import stats

from astropy import units as u
from astropy.coordinates import Distance
from astropy.table import Table

In [ ]:
%matplotlib inline
from matplotlib import style, pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

plt.rcParams['image.cmap'] = 'viridis'
plt.rcParams['image.origin'] = 'lower'
plt.rcParams['axes.prop_cycle'] = style.library['seaborn-deep']['axes.prop_cycle']
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['axes.titlesize'] =  plt.rcParams['axes.labelsize'] = 16
plt.rcParams['xtick.labelsize'] =  plt.rcParams['ytick.labelsize'] = 14

# Load Data

In [ ]:
meanstack_fns = {'draco': ('draco_mean_db_jegpeek.fits.gz', 'stack_draco_jegpeek.fits.gz')}
for i in range(5):
    mnfn = 'vdev_{:02.0f}_mn_db_jegpeek.fits.gz'.format(i)
    stfn = 'vdev_{:02.0f}_st_db_jegpeek.fits.gz'.format(i)
    meanstack_fns['vdev_{:02.0f}'.format(i)] = (mnfn, stfn)

In [ ]:
meanstack_tabs = {}
for nm, (mean_fn, stack_fn) in meanstack_fns.items():
    mean_tab = Table.read(mean_fn)
    stack_tab = Table.read(stack_fn)
    meanstack_tabs[nm] = (mean_tab, stack_tab)

In [ ]:
draco_d = Distance(80, u.kpc)
Distance([200, 400, 1500], u.kpc).distmod - draco_d.distmod

# Star/Galaxy Separation 

In [ ]:
def stargalprob(r_kron, r_kron_err, r_ap, r_ap_err, r_psf, r_psf_err, width=.2):
    kronpsferr = np.hypot(r_kron_err, r_psf_err)
    appsferr = np.hypot(r_ap_err, r_psf_err)
    
    dkron = np.abs(r_kron-r_psf)/kronpsferr
    dap = np.abs(r_ap-r_psf)/appsferr
    dmean = (dkron + dap)/2
    
    #return stats.norm.cdf(-dap,0,1)*2
    return stats.norm.cdf(-dap*appsferr,0, width)*2

In [ ]:
for mean_tab, stack_tab in meanstack_tabs.values():
    mean_tab['starprob'] = stargalprob(mean_tab['m_rMeanKronMag'], mean_tab['m_rMeanKronMagErr'],
                                       mean_tab['m_rMeanApMag'], mean_tab['m_rMeanApMagErr'],
                                       mean_tab['m_rMeanPSFMag'], mean_tab['m_rMeanPSFMagErr'], width=.1)
    stack_tab['starprob'] = stargalprob(stack_tab['st_rKronMag'], stack_tab['st_rKronMagErr'],
                                        stack_tab['st_rApMag'], stack_tab['st_rApMagErr'],
                                        stack_tab['st_rPSFMag'], stack_tab['st_rPSFMagErr'], width=.3)

# Randomize and show 

In [ ]:
randomized_order = np.random.permutation(list(meanstack_tabs.keys()))

In [ ]:
for i, nm in enumerate(randomized_order):
    mean_tab, stack_tab = meanstack_tabs[nm]
    
    plt.figure()
    
    gmi_stack = stack_tab['st_gPSFMag'] - stack_tab['st_iPSFMag']
    r_stack = stack_tab['st_rPSFMag']
    starmsk_stack = stack_tab['starprob'] >0.5

    gmi_mean = mean_tab['m_gMeanPSFMag'] - mean_tab['m_iMeanPSFMag']
    r_mean = mean_tab['m_rMeanPSFMag']
    starmsk_mean = mean_tab['starprob'] >0.5

    fig, axs = plt.subplots(1, 2)
    axs[0].scatter(gmi_mean[starmsk_mean], r_mean[starmsk_mean], alpha=.25, lw=0, s=2, c='r')
    axs[0].scatter(gmi_mean[~starmsk_mean], r_mean[~starmsk_mean], alpha=.1, lw=0, s=1, c='k')
    axs[1].scatter(gmi_stack[starmsk_stack], r_stack[starmsk_stack], alpha=.25, lw=0, s=2, c='b')
    axs[1].scatter(gmi_stack[~starmsk_stack], r_stack[~starmsk_stack], alpha=.1, lw=0, s=1, c='k')

    for ax in axs:
        ax.set_xlim(-.5, 1.8)
        ax.set_ylim(23.2, 15)

        ax.set_xlabel('g-i')
        ax.set_ylabel('r')

    axs[0].set_title('mean, i={}'.format(i))
    axs[1].set_title('stack, i={}'.format(i))
    
    
print('#Paste this in the cell below for final judgements *before* revealing the names:')
print('sat_present = {}')
for i in range(len(randomized_order)):
    print("sat_present[{}] = 'unknown' # 'yes', 'no', 'maybe'".format(i))
print('sat_present')
print("""
for i, nm in  enumerate(randomized_order):
    print(nm,':', sat_present[i])
"""[1:-1])

In [ ]:
sat_present = {}
sat_present[0] = 'yes' # 'yes', 'no', 'maybe'
sat_present[1] = 'no' # 'yes', 'no', 'maybe'
sat_present[2] = 'no' # 'yes', 'no', 'maybe'
sat_present[3] = 'maybe' # 'yes', 'no', 'maybe'
sat_present[4] = 'no' # 'yes', 'no', 'maybe'
sat_present[5] = 'no' # 'yes', 'no', 'maybe'
sat_present
for i, nm in  enumerate(randomized_order):
    print(nm,':', sat_present[i])

# Look more closely at maybe's/yes's 

In [ ]:
def exam_plot(nm, rad=10*u.arcmin, alphascale=1, sizescale=1, bgcolor=None, show_far=True):
    mean_tab, stack_tab = meanstack_tabs[nm]

    gmi_stack = stack_tab['st_gPSFMag'] - stack_tab['st_iPSFMag']
    r_stack = stack_tab['st_rPSFMag']
    starmsk_stack = stack_tab['starprob'] >0.5

    gmi_mean = mean_tab['m_gMeanPSFMag'] - mean_tab['m_iMeanPSFMag']
    r_mean = mean_tab['m_rMeanPSFMag']
    starmsk_mean = mean_tab['starprob'] >0.5

    fig, axs = plt.subplots(2, 2, figsize=(14,14))

    axs.flat[0].scatter(gmi_mean[starmsk_mean], r_mean[starmsk_mean], alpha=.2*alphascale, lw=0, s=2*sizescale, c='r')
    axs.flat[0].scatter(gmi_mean[~starmsk_mean], r_mean[~starmsk_mean], alpha=.1, lw=0, s=1, c='k')

    axs.flat[1].scatter(gmi_stack[starmsk_stack], r_stack[starmsk_stack], alpha=.2*alphascale, lw=0, s=2*sizescale, c='b')
    axs.flat[1].scatter(gmi_stack[~starmsk_stack], r_stack[~starmsk_stack], alpha=.1, lw=0, s=1, c='k')

    ra0 = np.mean(mean_tab['o_raMean'])
    dec0 = np.mean(mean_tab['o_decMean'])
    near_mean = np.hypot((mean_tab['o_raMean']-ra0)*np.cos(dec0*u.deg), mean_tab['o_decMean']-dec0)*u.deg < rad
    axs.flat[2].scatter(gmi_mean[near_mean], r_mean[near_mean], alpha=.5*alphascale, lw=0, s=1*sizescale, c=1-mean_tab['starprob'][near_mean])
    if show_far:
        axs.flat[2].scatter(gmi_mean[~near_mean], r_mean[~near_mean], alpha=.1, lw=0, s=1, c='k')

    ra0 = np.mean(stack_tab['st_raStack'])
    dec0 = np.mean(stack_tab['st_decStack'])
    near_stack = np.hypot((stack_tab['st_raStack']-ra0)*np.cos(dec0*u.deg), stack_tab['st_decStack']-dec0)*u.deg < rad
    axs.flat[3].scatter(gmi_stack[near_stack], r_stack[near_stack], alpha=.5*alphascale, lw=0, s=1*sizescale, c=1-stack_tab['starprob'][near_stack])
    if show_far:
        axs.flat[3].scatter(gmi_stack[~near_stack], r_stack[~near_stack], alpha=.1, lw=0, s=1, c='k')

    for ax in axs.flat:
        if bgcolor is not None:
            ax.set_axis_bgcolor(bgcolor)
        ax.set_xlim(-.5, 1.8)
        ax.set_ylim(23.2, 15)

        ax.set_xlabel('g-i')
        ax.set_ylabel('r')

    axs.flat[0].set_title('mean, {}'.format(nm))
    axs.flat[1].set_title('stack, {}'.format(nm))

In [ ]:
exam_plot('draco', 10*u.arcmin, bgcolor=None)

In [ ]:
exam_plot('vdev_00', 5*u.arcmin, sizescale=2, show_far=False)

# Appendices

# S/G separation checks

## Ap 

In [ ]:
for i, nm in enumerate(randomized_order):
    mean_tab, stack_tab = meanstack_tabs[nm]
    
    fig, axs = plt.subplots(1, 2)
    
    band = 'r'
    nmax = 300000
    mean_sample = np.random.permutation(len(mean_tab))[:nmax]
    stack_sample = np.random.permutation(len(stack_tab))[:nmax]
    
    
    other = mean_tab['m_{}MeanApMag'.format(band)][mean_sample]
    psf = mean_tab['m_{}MeanPSFMag'.format(band)][mean_sample]
    sc = axs[0].scatter(psf, psf-other, alpha=.2, lw=0, s=2, c=mean_tab['starprob'][mean_sample])

    other = stack_tab['st_{}ApMag'.format(band)][stack_sample]
    psf = stack_tab['st_{}PSFMag'.format(band)][stack_sample]
    sc = axs[1].scatter(psf, psf-other, alpha=.2, lw=0, s=2, c=stack_tab['starprob'][stack_sample])
    
    axs[0].set_title('mean, i={}'.format(i))
    axs[1].set_title('stack, i={}'.format(i))
    
    for ax in axs:
        ax.set_xlabel('PSF')
        ax.set_ylabel('PSF - Ap')
        ax.set_ylim(-1.0, 1.0)
        ax.set_xlim(15, 23)
        
    plt.colorbar(sc).set_label(r'$P_{\rm star}$')

## Kron

In [ ]:
for i, nm in enumerate(randomized_order):
    mean_tab, stack_tab = meanstack_tabs[nm]
    
    fig, axs = plt.subplots(1, 2)
    
    band = 'r'
    nmax = 300000
    mean_sample = np.random.permutation(len(mean_tab))[:nmax]
    stack_sample = np.random.permutation(len(stack_tab))[:nmax]
    
    
    other = mean_tab['m_{}MeanKronMag'.format(band)][mean_sample]
    psf = mean_tab['m_{}MeanPSFMag'.format(band)][mean_sample]
    sc = axs[0].scatter(psf, psf-other, alpha=.2, lw=0, s=2, c=mean_tab['starprob'][mean_sample])

    other = stack_tab['st_{}KronMag'.format(band)][stack_sample]
    psf = stack_tab['st_{}PSFMag'.format(band)][stack_sample]
    sc = axs[1].scatter(psf, psf-other, alpha=.2, lw=0, s=2, c=stack_tab['starprob'][stack_sample])
    
    axs[0].set_title('mean, i={}'.format(i))
    axs[1].set_title('stack, i={}'.format(i))
    
    for ax in axs:
        ax.set_xlabel('PSF')
        ax.set_ylabel('PSF - Kron')
        ax.set_ylim(-1.0, 1.0)
        ax.set_xlim(15, 23)
        
    plt.colorbar(sc).set_label(r'$P_{\rm star}$')

# In different bands 

# Kron vs PSF

In [ ]:
for i, nm in enumerate(randomized_order):
    mean_tab, stack_tab = meanstack_tabs[nm]
    
    fig, axs = plt.subplots(1, 2)
    for band, color in zip('gri', 'grm'):
        other = mean_tab['m_{}MeanKronMag'.format(band)]
        psf = mean_tab['m_{}MeanPSFMag'.format(band)]
        axs[0].scatter(psf, psf-other, alpha=.05, lw=0, s=2, c=color)
        
        other = stack_tab['st_{}KronMag'.format(band)]
        psf = stack_tab['st_{}PSFMag'.format(band)]
        axs[1].scatter(psf, psf-other, alpha=.05, lw=0, s=2, c=color)
    
    axs[0].set_title('mean, i={}'.format(i))
    axs[1].set_title('stack, i={}'.format(i))
    
    for ax in axs:
        ax.set_xlabel('PSF')
        ax.set_ylabel('PSF - Kron')
        ax.set_ylim(-1.0, 1.0)
        ax.set_xlim(15, 23)
    

# S/G Aperture vs PSF

In [ ]:
for i, nm in enumerate(randomized_order):
    mean_tab, stack_tab = meanstack_tabs[nm]
    
    fig, axs = plt.subplots(1, 2)
    for band, color in zip('gri', 'grm'):
        other = mean_tab['m_{}MeanApMag'.format(band)]
        psf = mean_tab['m_{}MeanPSFMag'.format(band)]
        axs[0].scatter(psf, psf-other, alpha=.05, lw=0, s=2, c=color)
        
        other = stack_tab['st_{}ApMag'.format(band)]
        psf = stack_tab['st_{}PSFMag'.format(band)]
        axs[1].scatter(psf, psf-other, alpha=.05, lw=0, s=2, c=color)
    
    axs[0].set_title('mean, i={}'.format(i))
    axs[1].set_title('stack, i={}'.format(i))
    
    for ax in axs:
        ax.set_xlabel('PSF')
        ax.set_ylabel('PSF - Ap')
        ax.set_ylim(-1.5, 1.5)
        ax.set_xlim(15, 23)
    

# Uncertainties 

In [ ]:
for i, nm in enumerate(randomized_order):
    mean_tab, stack_tab = meanstack_tabs[nm]
    
    fig, axs = plt.subplots(1, 2)
    for band, color in zip('gri', 'grm'):
        psf = mean_tab['m_{}MeanPSFMag'.format(band)]
        psf_err = mean_tab['m_{}MeanPSFMagErr'.format(band)]
        axs[0].scatter(psf, psf_err, alpha=.05, lw=0, s=2, c=color)
        
        psf = stack_tab['st_{}PSFMag'.format(band)]
        psf_err = stack_tab['st_{}PSFMagErr'.format(band)]
        axs[1].scatter(psf, psf_err, alpha=.05, lw=0, s=2, c=color)
    
    axs[0].set_title('mean, i={}'.format(i))
    axs[1].set_title('stack, i={}'.format(i))
    
    for ax in axs:
        ax.set_xlabel('PSF')
        ax.set_ylabel('PSF_err')
        ax.set_ylim(0, .5)
        ax.set_xlim(15, 23)
    

In [ ]:
for i, nm in enumerate(randomized_order):
    mean_tab, stack_tab = meanstack_tabs[nm]
    
    fig, axs = plt.subplots(1, 2)
    for band, color in zip('gri', 'grm'):
        psf = mean_tab['m_{}MeanKronMag'.format(band)]
        psf_err = mean_tab['m_{}MeanKronMagErr'.format(band)]
        axs[0].scatter(psf, psf_err, alpha=.05, lw=0, s=2, c=color)
        
        psf = stack_tab['st_{}KronMag'.format(band)]
        psf_err = stack_tab['st_{}KronMagErr'.format(band)]
        axs[1].scatter(psf, psf_err, alpha=.05, lw=0, s=2, c=color)
    
    axs[0].set_title('mean, i={}'.format(i))
    axs[1].set_title('stack, i={}'.format(i))
    
    for ax in axs:
        ax.set_xlabel('Kron')
        ax.set_ylabel('Kron_err')
        ax.set_ylim(0, .5)
        ax.set_xlim(15, 23)
    